# organoid Notebook

In [ ]:
from organoid import segmentation
from organoid import preprocessing
from organoid import reconstruction

import tifffile

Example of usage of segmentation functions

In [ ]:
path = rf'C:\Users\gros\Desktop\DATA\organoid_test'
data = tifffile.imread(rf'{path}\data.tif')
dapi=data[:,0,:,:]
stardist_model = rf'C:\Users\gros\Desktop\CODES\Alice_Segmentation\Stardist\models\lennedist_3d_grid222_rays64'

In [ ]:
output = segmentation.predict_stardist(data,model_path=stardist_model,input_voxelsize=[1,1,1],normalize_input=True)

Example of usage of reconstruction functions

In [ ]:
path = rf'Z:\EqpLenne2\Alice\Lecuit_biphoton\13072023_divrates_FGFA\72h'
reconstruction.plot_positions(path_bottom_positions=rf'{path}\multipoints_bottom.xml',
                              path_top_positions=rf'{path}\multipoints_top.xml')

In [ ]:
path = rf'C:\Users\gros\Desktop\DATA\organoid_test\Registration'
path_to_bin = rf'C:\Users\gros\Anaconda3\envs\registration-env\Library\bin'
list_bottom=[1] #the length of this list corresponds to the number of samples.
list_top=[1]
channels=['dapi','ph3','bra','ecad']

In [ ]:
#Creates folder structure
reconstruction.create_folders(path,list_bottom,list_top,channels)

In [ ]:
#from now on, we consider only one sample. If you have multiple samples, you can loop over the parameter 'sample_id'

#register the reference channel first
reconstruction.register(path,path_to_bin=path_to_bin,sample_id='1',channel='dapi',input_voxel=[0.62,0.62,2],
                        output_voxel=[1,1,1],compute_trsf=1,init_trsfs=[["flip", "Y", "flip", "Z", "trans", "Z", -10]],
                        trsf_type='rigid',depth=3,save_json=True)

#and the others using the same transformation
for channel in channels[1:]:
    print(channel)
    reconstruction.register(path,path_to_bin=path_to_bin,sample_id='1',channel=channel,input_voxel=[0.62,0.62,2],
                            output_voxel=[1,1,1],compute_trsf=0,save_json=False)


In [ ]:
#Visualising the result
reconstruction.check_napari(path, sample_id= '1',channel = 'ecad')

In [ ]:
#Fuse the 2 registered sides into one array
reconstruction.fuse_sides(path,sample_id='1', channels=channels)

In [ ]:
#Fuse all the channels in one image
reconstruction.write_hyperstacks(path, sample_id='1',channels=channels)